### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
today

datetime.date(2023, 2, 19)

### Process after last business day, yesterday must be last business day

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-02-19
yesterday: 2023-02-17 00:00:00


In [3]:
yesterday = yesterday.date()
week_ago = yesterday -timedelta(days = 7)
print(f'a week ago: {week_ago}')
print(f'yesterday: {yesterday}')

a week ago: 2023-02-10
yesterday: 2023-02-17


### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [4]:
cols = 'name price_w price_d percent perform mrkt '.split()

format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [5]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,ASK,2023-02-16,31.00


In [6]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'ASK'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,38,ASK,SET,32.75,49.50,28.25,11.97,1.75,"2,639.22","17,286.87",50.11,1.75,38,2017-07-23 07:24:24.813799,2023-01-31 03:07:59.778346


In [7]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "ASK" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,ASK,2023-02-17,30.25,31.00,29.25,"3,652,773",31.00
1,ASK,2023-02-16,31.00,31.75,30.75,"3,653,445",31.25
2,ASK,2023-02-15,30.75,32.50,30.50,"2,500,112",32.25
3,ASK,2023-02-14,32.25,33.00,32.00,"715,070",32.75
4,ASK,2023-02-13,32.75,33.00,32.50,"315,216",32.50


In [8]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (yesterday, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2023-02-17' AND name IN ('ASK', 'ASP', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART')
ORDER BY name, date


26

In [9]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (week_ago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2023-02-10' AND name IN ('ASK', 'ASP', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART') 
ORDER BY name


26

In [10]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(26, 3)

In [11]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [12]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [13]:
trend

,name,price_d,price_w,percent
0,ASK,30.25,32.50,-6.92
1,ASP,3.04,3.12,-2.56
2,BANPU,10.90,10.90,0.00
3,BCH,21.20,20.80,1.92
4,CPNREIT,18.30,18.40,-0.54
5,DIF,13.30,13.30,0.00
6,GVREIT,9.70,9.65,0.52
7,IVL,38.75,40.50,-4.32
8,JASIF,8.15,8.20,-0.61
9,JMART,27.25,34.50,-21.01


In [14]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [15]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
19,SINGER,17.30,26.75,-35.33%,Worse
9,JMART,27.25,34.50,-21.01%,Worse
10,JMT,45.75,50.50,-9.41%,Worse
0,ASK,30.25,32.50,-6.92%,Worse
13,NER,5.85,6.25,-6.40%,Worse
16,RCL,29.25,30.75,-4.88%,Worse
7,IVL,38.75,40.50,-4.32%,Worse
15,PTTGC,48.75,50.50,-3.47%,Worse
11,KCE,46.00,47.25,-2.65%,Worse
1,ASP,3.04,3.12,-2.56%,Worse


### Filter only performance = "Worse"

In [16]:
mask = trend.perform == 'Worse'
trend[mask]

,name,price_d,price_w,percent,perform
0,ASK,30.25,32.50,-6.92,Worse
1,ASP,3.04,3.12,-2.56,Worse
4,CPNREIT,18.30,18.40,-0.54,Worse
7,IVL,38.75,40.50,-4.32,Worse
8,JASIF,8.15,8.20,-0.61,Worse
9,JMART,27.25,34.50,-21.01,Worse
10,JMT,45.75,50.50,-9.41,Worse
11,KCE,46.00,47.25,-2.65,Worse
13,NER,5.85,6.25,-6.40,Worse
14,ORI,11.80,11.90,-0.84,Worse


In [17]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,76.92%
No Change,11.54%
Better,11.54%


In [18]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(253, 8)

In [19]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [20]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(26, 13)

In [21]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
9,JMART,34.50,27.25,-21.01%,Worse,SET50
10,JMT,50.50,45.75,-9.41%,Worse,SET50
7,IVL,40.50,38.75,-4.32%,Worse,SET50
15,PTTGC,50.50,48.75,-3.47%,Worse,SET50
2,BANPU,10.90,10.90,0.00%,No Change,SET50
17,SCC,338.00,340.00,0.59%,Better,SET50


In [22]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,66.67%
No Change,16.67%
Better,16.67%


In [23]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
19,SINGER,26.75,17.30,-35.33%,Worse,SET100
16,RCL,30.75,29.25,-4.88%,Worse,SET100
11,KCE,47.25,46.00,-2.65%,Worse,SET100
14,ORI,11.90,11.80,-0.84%,Worse,SET100
20,STA,23.90,23.80,-0.42%,Worse,SET100
3,BCH,20.80,21.20,1.92%,Better,SET100


In [24]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,83.33%
Better,16.67%


In [25]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
0,ASK,32.50,30.25,-6.92%,Worse,SET999
13,NER,6.25,5.85,-6.40%,Worse,SET999
1,ASP,3.12,3.04,-2.56%,Worse,SET999
25,WHART,11.70,11.40,-2.56%,Worse,SET999
22,TFFIF,8.15,7.95,-2.45%,Worse,SET999
23,TMT,8.20,8.00,-2.44%,Worse,SET999
21,SYNEX,16.50,16.10,-2.42%,Worse,SET999
24,WHAIR,7.90,7.85,-0.63%,Worse,SET999
8,JASIF,8.20,8.15,-0.61%,Worse,SET999
4,CPNREIT,18.40,18.30,-0.54%,Worse,SET999


In [26]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,78.57%
No Change,14.29%
Better,7.14%


In [27]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt


In [28]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
